## Ejercicios propuestos

# Ejercicio 1

Ejercicio 1: Escribir una UDF que convierta a minusculas una columna, y se aplique para todas las columnas tipo String del dataframe ttgoficiDF mostrado a continuación:

In [2]:
import pyspark
from pyspark.sql.context import SQLContext
from pyspark.sql.types import *
sc = pyspark.SparkContext('local[*]') 
sqlContext = SQLContext(sc)
dataPath = "../data/"
customSchema = StructType([  
 StructField("cod_bancsb",  StringType(), True),
 StructField("cod_ofici",  IntegerType(), True),
 StructField("cnivel",  StringType(), True),
 StructField("cod_zona",  StringType(), True),
 StructField("cod_territor",  StringType(), True),
 StructField("cod_dirgener",  StringType(), True),
 StructField("cod_areanego",  IntegerType(), True),
 StructField("cod_dar",  StringType(), True),
 StructField("des_nomco",  StringType(), True),
 StructField("des_nomab",  StringType(), True),
 StructField("f_cierre",  StringType(), True),
 StructField("cod_cbc",  StringType(), True)])

ttgoficiDF = sqlContext.read.format("com.databricks.spark.csv")\
            .option("header", "true")\
            .load(dataPath + "ttgofici.csv", schema=customSchema)            

In [3]:
ttgoficiDF.collect()

[Row(cod_bancsb='9', cod_ofici=555, cnivel='10', cod_zona='0', cod_territor='4744', cod_dirgener='0', cod_areanego=9007, cod_dar='0', des_nomco='CENTRALIZADORA OFICINAS (FINANZIA)', des_nomab='CTDRA.OFNAS.FINANZIA', f_cierre='2011-07-11', cod_cbc='0'),
 Row(cod_bancsb='9', cod_ofici=557, cnivel='10', cod_zona='0', cod_territor='3860', cod_dirgener='0', cod_areanego=9007, cod_dar='0', des_nomco='CENTRALIZADORA FACTORING', des_nomab='CENTRALIZ FACTORING', f_cierre='2011-07-11', cod_cbc='0'),
 Row(cod_bancsb='9', cod_ofici=574, cnivel='10', cod_zona='0', cod_territor='4744', cod_dirgener='0', cod_areanego=9007, cod_dar='0', des_nomco='OF. MOAD ADJUDICACION DE INMUEBLES', des_nomab='OF.MOAD ADJUDIC. INM', f_cierre='2011-07-11', cod_cbc='0'),
 Row(cod_bancsb='9', cod_ofici=991, cnivel='10', cod_zona='0', cod_territor='3860', cod_dirgener='0', cod_areanego=9007, cod_dar='0', des_nomco='CENTRO ADMINISTRATI. REGIONAL MADRID', des_nomab='CTRO.ADM.REG.MADRID', f_cierre='1991-10-17', cod_cbc='0')

In [4]:
def lower(string):
    return string.lower()

In [5]:
from pyspark.sql.functions import col,udf
udLower = udf(lower, StringType())
ttgoficiDF.withColumn("des_nomab_lower",udLower("des_nomab")).show()

+----------+---------+------+--------+------------+------------+------------+-------+--------------------+--------------------+----------+-------+--------------------+
|cod_bancsb|cod_ofici|cnivel|cod_zona|cod_territor|cod_dirgener|cod_areanego|cod_dar|           des_nomco|           des_nomab|  f_cierre|cod_cbc|     des_nomab_lower|
+----------+---------+------+--------+------------+------------+------------+-------+--------------------+--------------------+----------+-------+--------------------+
|         9|      555|    10|       0|        4744|           0|        9007|      0|CENTRALIZADORA OF...|CTDRA.OFNAS.FINANZIA|2011-07-11|      0|ctdra.ofnas.finanzia|
|         9|      557|    10|       0|        3860|           0|        9007|      0|CENTRALIZADORA FA...| CENTRALIZ FACTORING|2011-07-11|      0| centraliz factoring|
|         9|      574|    10|       0|        4744|           0|        9007|      0|OF. MOAD ADJUDICA...|OF.MOAD ADJUDIC. INM|2011-07-11|      0|of.moad adjudi

In [1]:
ttgoficiDF.schema

NameError: name 'ttgoficiDF' is not defined

# Ejercicio 2

Ejercicio 2: Realizara una "UDF" que convierta el primer caracter de un string según la siguiente regla:  
mapping = {'1': 'a', '2': '1', '3': '2', '4': '3', '5': '4', '6': '5', '7': '6', '8':'7'}  
'1'-> 'a'  
'2'-> '1'  
'3'-> '2'  
'4'-> '3'  
'5'-> '4'  
'6'-> '5'  
'7'-> '6'  
'8'-> '7'  
De tal forma un String con valor: "1.Semana del 1 al 7 de agosto" se convertirá a : "a..Semana del 1 al 7 de   agosto", "2.Semana del 8 al 14 de agosto" se convertirá a "1.Semana del 8 al 14 de agosto"... y así sucesivamente.  
Tip: utilizar el siguiente mapping:  
mapping = {'1': 'a', '2': '1', '3': '2', '4': '3', '5': '4', '6': '5', '7': '6', '8':'7'}

In [14]:
data = [('1', '1.Semana del 1 al 7 de agosto'),('2', '2.Semana del 8 al 14 de agosto'),\
     ('3', '3.Semana del 15 al 21 de agosto'),('4', '3.Semana del 15 al 21 de agosto')]
DFSpentData =  sqlContext.createDataFrame(data, ['id','Semana'])

In [15]:
DFSpentData.show()

+---+--------------------+
| id|              Semana|
+---+--------------------+
|  1|1.Semana del 1 al...|
|  2|2.Semana del 8 al...|
|  3|3.Semana del 15 a...|
|  4|3.Semana del 15 a...|
+---+--------------------+



In [16]:
mapping = {'1': 'a', '2': '1', '3': '2', '4': '3', '5': '4', '6': '5', '7': '6', '8':'7'}

In [17]:
from pyspark.sql.functions import UserDefinedFunction
udfSemana = UserDefinedFunction(lambda x: mapping[x[0]] + x[1:], StringType())

In [18]:
DFSpentData.withColumn("semana2",udfSemana("semana")).collect()

[Row(id='1', Semana='1.Semana del 1 al 7 de agosto', semana2='a.Semana del 1 al 7 de agosto'),
 Row(id='2', Semana='2.Semana del 8 al 14 de agosto', semana2='1.Semana del 8 al 14 de agosto'),
 Row(id='3', Semana='3.Semana del 15 al 21 de agosto', semana2='2.Semana del 15 al 21 de agosto'),
 Row(id='4', Semana='3.Semana del 15 al 21 de agosto', semana2='2.Semana del 15 al 21 de agosto')]

### In sql sentence...

In [24]:
sqlContext.registerDataFrameAsTable(DFSpentData,"SpentData")

In [25]:
sqlContext.registerFunction("getDay", lambda x: mapping[x[0]] + x[1:], StringType())

In [29]:
sqlContext.sql("select id, udfSemana(Semana) as semana3 from SpentData").collect()

[Row(id='1', semana3='a.Semana del 1 al 7 de agosto'),
 Row(id='2', semana3='1.Semana del 8 al 14 de agosto'),
 Row(id='3', semana3='2.Semana del 15 al 21 de agosto'),
 Row(id='4', semana3='2.Semana del 15 al 21 de agosto')]

# Ejercicio 3 

Ejercicio 3: Migrar las siguientes transformaciones en lenguaje a SAS a SparkSQL:

data inframul_diaria;  
	set diaria.Nuevanet_di_con;  

	dia = substr(FEC_ULTACCSO,9,2);  
	mes = substr(FEC_ULTACCSO,6,2);  
	anio= substr(FEC_ULTACCSO,1,4);  
	fecha = compress(anio||mes||dia);  
	format fecha_ult_conexion_semanal date9.;  
	fecha_ult_conexion_semanal =input(fecha, yymmdd10.);  
	cod_persona = input (COD_PERSCTPN, 11.);  
run;

proc sql;  
	create table cli_net_semanal as  
	select distinct cod_persona,  
			max (fecha_ult_conexion_semanal) as fecha_ult_conexion_semanal format date9.  
	from inframul_diaria  
	where fecha_ult_conexion_semanal>=&FEC_INI02  
	group by cod_persona 
	order by cod_persona;  
quit;


In [ ]:
query = "Select *, cod_persctpn as cod_persona FROM nuevanet_di_con"
udf_dia = UserDefinedFunction(lambda x: x[8:10], StringType())
udf_mes = UserDefinedFunction(lambda x: x[5:7], StringType())
udf_anio = UserDefinedFunction(lambda x: x[0:4], StringType())
udf_fecha = UserDefinedFunction(lambda x,y,z: x+y+z, StringType())
udf_fecha_mes = UserDefinedFunction(lambda x,y,z: x+mes_l[y]+z, StringType())
campo_fecha = 'fec_ultaccso'

sqlContext.sql(query).withColumn("dia",udf_dia(campo_fecha))\
.withColumn("mes",udf_mes(campo_fecha))\
.withColumn("anio",udf_anio(campo_fecha))\
.withColumn("fecha",udf_fecha("dia","mes","anio"))\
.withColumn("fecha_ult_conexion_semanal",udf_fecha_mes("dia","mes","anio"))\
.registerTempTable("inframul_diaria")

FEC_INI02 = "01ENE2000"
sqlContext.sql("Select distinct cod_persona, max(fecha_ult_conexion_semanal) as fecha_ult_conexion_semanal \
FROM inframul_diaria \
WHERE  fecha_ult_conexion_semanal>='" + FEC_INI02 + "'\
GROUP BY cod_persona")\
.registerTempTable("cli_net_semanal")